In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

class Main(webdriver.Chrome):
  def __init__(self):
    options = webdriver.ChromeOptions().add_experimental_option("excludeSwitches", ["enable-logging"])
    service = Service(ChromeDriverManager().install())
    super(Main, self).__init__(service=service, options=options)
    self.maximize_window()
    self.get("https://voluntarias.plataformamaisbrasil.gov.br/voluntarias/Principal/Principal.do?Usr=guest&Pwd=guest")
    
  def readSingleColumn(self, columnName, lines):
    df = pd.read_excel("/home/adriano/Downloads/solicitacoes.xlsx")
    lines = df.iloc[1:lines]
    return lines[columnName].to_list()
  
  def writeDocument(self, data, colunas):
    df = pd.DataFrame(data).transpose()
    df.columns = colunas
    df.to_excel('cnpj.xlsx', index=False, header=True)
    

  
  def getData(self, array):
    self.implicitly_wait(30)
    self.find_element(By.CSS_SELECTOR, '.col1 :nth-child(4)').click()
    self.find_element(By.CSS_SELECTOR, '.colM ul > li a' ).click()
    search_url = self.current_url
    
    data = [[],[]]
    for convenio in array:
      self.implicitly_wait(10)
      self.find_element(By.CSS_SELECTOR, 'input[id=consultarNumeroConvenio]').send_keys(convenio)
      self.find_element(By.CSS_SELECTOR, 'input[id=form_submit][type=button]').click()
      self.find_element(By.CSS_SELECTOR, '.numeroConvenio a').click()
      cnpj = self.find_element(By.CSS_SELECTOR, '.proponente .field > div').text
      data[0].append(convenio)
      data[1].append(cnpj.split(' ')[1])
      self.get(search_url)
      self.find_element(By.CSS_SELECTOR, 'input[id=consultarNumeroConvenio]').clear()
      self.find_element(By.CSS_SELECTOR, 'input[id=consultarNumeroProposta]').clear()
    self.writeDocument(data, ["convenio", "cnpj"])
    self.close()
    
  
  
bot = Main()
requestList = bot.readSingleColumn("N° Convênio", 10)
bot.getData(requestList)


ModuleNotFoundError: No module named 'webdriver_manager'